In [1]:
## it’s mine, my own, 
from modules.get_data import get_x_y
from jh_utils.data.pandas.preprocessing import make_dummies

ModuleNotFoundError: No module named 'modules'

In [7]:
## third
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## models
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
## xgboost
from sklearn.linear_model import LogisticRegression

## pos model
from sklearn.metrics import classification_report,confusion_matrix

In [8]:
x,y = get_x_y_no_NaNs()

/home/joaoholz/Documents/ufes/trabalho_final/modules/get_data.py:5: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv('asteroides/asteoides.csv').iloc[:,first_columns:]


In [9]:
x

,neo,tp_cal,tp,class,w,om,i,q,a,e,n,epoch_cal,epoch_mjd,epoch
0,0,2.018043e+07,2.458239e+06,MBA,73.597695,80.305531,10.594067,2.558684,2.769165,0.076009,0.213885,20190427.0,58600,2458600.5
1,0,2.018072e+07,2.458321e+06,MBA,310.202392,173.024741,34.832932,2.135935,2.773841,0.229972,0.213345,20200531.0,59000,2459000.5
2,0,2.018112e+07,2.458446e+06,MBA,248.066193,169.851482,12.991043,1.982706,2.668285,0.256936,0.226129,20200531.0,59000,2459000.5
3,0,2.018051e+07,2.458248e+06,MBA,150.728541,103.810804,7.141771,2.151909,2.361418,0.088721,0.271609,20190427.0,58600,2458600.5
4,0,2.020032e+07,2.458926e+06,MBA,358.648418,141.571026,5.367427,2.082619,2.574037,0.190913,0.238661,20200531.0,59000,2459000.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958519,0,1.960113e+07,2.437266e+06,MBA,195.442802,194.229902,5.984416,1.856807,2.280861,0.185919,0.286125,19600926.0,37203,2437203.5
958520,0,2.021053e+07,2.459362e+06,MBA,5.216223,355.248239,8.082280,1.674319,2.334910,0.282920,0.276248,20200531.0,59000,2459000.5
958521,1,2.021123e+07,2.459574e+06,APO,234.894852,182.982832,4.679278,0.950941,2.817152,0.662446,0.208444,20200531.0,59000,2459000.5
958522,0,1.973102e+07,2.441974e+06,MBA,198.026527,176.499082,0.732484,1.893638,2.373137,0.202053,0.269600,19731001.0,41956,2441956.5


In [41]:
x = pd.concat([x,make_dummies(x['class'])],axis=1)
x.drop('class',axis=1,inplace=True)

In [42]:
y = make_dummies(y)

---
model

In [177]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,stratify=y)

In [178]:
standard = StandardScaler()
x_train = standard.fit_transform(x_train)
x_test = standard.transform(x_test)

In [137]:
def fit_classify(model):
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    
    print(classification_report(y_test,y_pred))
    cm = confusion_matrix(y_test,y_pred)
    col = ['N','Y']
    print(pd.DataFrame(cm,index=col,columns = col))

In [138]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

In [179]:
## optimizer
rms='rmsprop'
adam='adam'

## loss
SPC = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
CC = tf.keras.losses.CategoricalCrossentropy()
BC = 'binary_crossentropy'
MSE = 'mse'
SCC = 'sparse_categorical_crossentropy'

## last layer
softmax = 'softmax'
sigmoid = 'sigmoid'


dropou x 2 layers x activation last layer

loss | opt | ll act | metric | dropout | layers
---|---|---|---|---|---
BC | rms | sigmoid | 143/620  | True | 2 layers
mse | adam | sigmoid | 134/620  | True | 2 layers
BC | adam | sigmoid | 49/620 | True| 2 layers
BC | adam | sigmoid | 58/620 | True | 2 layers
BC | adam | sigmoid | 12/620 >0.025  |True | 2 layers
BC | adam | sigmoid | 78-54/620 | no dropout | 1 layer 
BC | adam | sigmoid | 26-5/620 | no dropout | 2 layers
BC | adam | sigmoid | 48-18/620 | no dropout | 3 layers


In [251]:
model_3 = tf.keras.Sequential()
model_3.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(25,)))
model_3.add(tf.keras.layers.Dense(16, activation='relu'))
model_3.add(tf.keras.layers.Dense(1, activation='sigmoid'))#sigmoid
model_3.compile(loss=BC, optimizer=adam, metrics=[tf.keras.metrics.Precision(),'accuracy'])

In [252]:
model_3.fit(x_train,y_train, epochs=8, batch_size=128, verbose=1)

Epoch 1/8
5133/5133 [==============================] - 14s 3ms/step - loss: 0.0164 - precision_7: 0.0021 - accuracy: 0.9964
Epoch 2/8
5133/5133 [==============================] - 13s 3ms/step - loss: 0.0065 - precision_7: 0.2500 - accuracy: 0.9978
Epoch 3/8
5133/5133 [==============================] - 14s 3ms/step - loss: 0.0063 - precision_7: 0.0000e+00 - accuracy: 0.9978
Epoch 4/8
5133/5133 [==============================] - 13s 3ms/step - loss: 0.0063 - precision_7: 0.0000e+00 - accuracy: 0.9978
Epoch 5/8
5133/5133 [==============================] - 13s 3ms/step - loss: 0.0062 - precision_7: 0.0000e+00 - accuracy: 0.9978
Epoch 6/8
5133/5133 [==============================] - 13s 3ms/step - loss: 0.0062 - precision_7: 0.0000e+00 - accuracy: 0.9978
Epoch 7/8
5133/5133 [==============================] - 13s 3ms/step - loss: 0.0061 - precision_7: 0.0000e+00 - accuracy: 0.9978
Epoch 8/8
5133/5133 [==============================] - 13s 3ms/step - loss: 0.0061 - precision_7: 0.0000e+00 - a

In [253]:
y_prob = model_3.predict(x_test)
y_prob.max()

8800/8800 [==============================] - 8s 861us/step


0.62538904

In [256]:
y_pred = list(map(lambda x: x>0.025, y_prob))
print(classification_report(y_test,y_pred))
cm = confusion_matrix(y_test,y_pred)
col = ['N','Y']
print(pd.DataFrame(cm,index=col,columns = col))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99    280961
           1       0.14      0.98      0.24       620

    accuracy                           0.99    281581
   macro avg       0.57      0.98      0.62    281581
weighted avg       1.00      0.99      0.99    281581

        N     Y
N  277104  3857
Y      14   606


In [9]:
rf = RandomForestClassifier(n_estimators=80)
fit_classify(rf)

/tmp/ipykernel_171056/2329049293.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train,y_train)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    187308
           1       0.61      0.06      0.10       413

    accuracy                           1.00    187721
   macro avg       0.80      0.53      0.55    187721
weighted avg       1.00      1.00      1.00    187721

        N   Y
N  187293  15
Y     390  23


In [10]:
dt = DecisionTreeClassifier()
fit_classify(dt)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    187308
           1       0.26      0.27      0.27       413

    accuracy                           1.00    187721
   macro avg       0.63      0.64      0.63    187721
weighted avg       1.00      1.00      1.00    187721

        N    Y
N  186991  317
Y     300  113
